# Imputace chybějících hodnot (Missing Value Imputation)


Chybějící hodnoty v datech mohou narušit analýzy a modely, proto je potřeba je vhodně doplnit (imputovat). Různé metody imputace nabízejí kompromis mezi jednoduchostí a přesností: od triviálních náhrad až po pokročilé modely využívající strojové učení. Volba metody závisí na povaze dat a důvodu, proč hodnoty chybí. Následuje přehled přístupů k imputaci chybějících hodnot:

- Jednoduché náhrady – Nahrazení chybějících hodnot konstantou nebo jednoduchou statistikou (např. nulou, průměrem, mediánem či módní hodnotou). Tyto metody jsou rychlé a snadno pochopitelné, ale mohou zkreslit rozdělení dat.

- Podobnostní metody – Imputace na základě podobných záznamů, například pomocí k-nejbližších sousedů (kNN) nebo shlukování. Chybějící hodnota se odhadne z hodnot záznamů, které jsou podle zvolené metriky vzdálenosti nejbližší neúplnému záznamu (případně se použije hodnota centroidu shluku).

- Prediktivní modely – Využití tradičních algoritmů strojového učení k predikci chybějící hodnoty z ostatních atributů. Lze použít rozhodovací stromy, náhodné lesy (např. MissForest algoritmus) či další regresní modely. Tyto modely se učí vztahům v datech a mohou přesněji odhadnout chybějící položky.

- Bayesovské přístupy – Statistické metody jako EM algoritmus (Expectation-Maximization) nebo vícenásobná imputace v řetězci (MICE) vytvářejí model distribučního rozdělení dat a opakovaně odhadují chybějící hodnoty na základě podmíněných rozdělení. Vícenásobná imputace generuje několik verzí doplněných datasetů, aby zohlednila nejistotu v odhadu.

- Hluboké učení – Moderní přístupy využívají hluboké neuronové sítě k imputaci. Autoenkodéry dokáží zakódovat záznam do latentního prostoru a následně rekonstruovat i chybějící hodnoty. Generativní modely jako VAE (variační autoenkodér) nebo GAN (generativní adversariální sítě, např. GAIN) se trénují tak, aby doplňovaly chybějící položky realisticky – např. GAN přístup s generátorem a diskriminátorem iterativně zlepšuje kvalitu imputací.

### ReMasker (Transformer MAE)
ReMasker je moderní algoritmus založený na principu maskovaného autoenkodéru s architekturou Transformer​. Hlavní myšlenkou je „remasking“, kdy se během trénování kromě skutečně chybějících hodnot náhodně zneviditelní i část pozorovaných hodnot. Model (maskovací autoenkodér) se optimalizuje k rekonstrukci těchto uměle zamlžených hodnot a následně se využije k predikci původně chybějících položek​​. Transformerová síť se samoorganizovanou pozorností tak dokáže zachytit složité vztahy mezi atributy v tabulce​. ReMasker nevyžaduje silné předpoklady o mechanismu chybění a funguje i při velmi vysokém podílu chybějících dat (autoři uvádějí úspěšné imputace i při 70 % chybějících hodnot)​. V rozsáhlém benchmarku na 12 datových sadách dosahuje ReMasker obdobné nebo lepší přesnosti než 13 známých metod, přičemž s rostoucím podílem chybějících dat se jeho náskok ještě zvyšuje​. 

[ReMasker repozitář](https://github.com/tydusky/remasker)

In [1]:
# TODO
# Remasker example
# This example shows how to use Remasker to create a mask for a given image.

import numpy as np
import pandas as pd
from utils import get_args_parser
from remasker_impute import ReMasker

X_raw = np.arange(50).reshape(10, 5) * 1.0
X = pd.DataFrame(X_raw, columns=['0', '1', '2', '3', '4'])
X.iat[3,0] = np.nan

imputer = ReMasker()

imputed = imputer.fit_transform(X)
print(imputed[3,0])


ModuleNotFoundError: No module named 'numpy'

### DeepIFSA 

DeepIFSA (Deep Imputation with Feature and Sample Attention) je framework, který využívá mechanismy pozornosti (attention) k imputaci tabulárních dat​. Model se učí samo-attention mezi vstupními rysy (sloupci) i mezi vzorky (řádky), čímž zachycuje vztahy jak mezi různými atributy, tak mezi různými záznamy. Inovativně je použit i trik z kontrastního učení – CutMix augmentace, kdy se části dat míchají, aby se zvýšila robustnost modelu​. DeepIFSA tak v podstatě kombinuje pozornost po sloupcích i řádcích v jedné hluboké síti. Podle výsledků autorů překonává tento přístup devět state-of-the-art metod v imputaci při různých typech chybění (MCAR, MAR, MNAR) a podílech chybějících dat 10–50 %​. Například ve zdravotnických EHR datech snížila pozornostní imputace normalizovanou RMSE o ~52–82 % oproti rozhodovacím stromečkům a dalším metodám​. Kód DeepIFSA je ve fázi výzkumného prototypu (autoři avizovali uvolnění na GitHubu)​

In [ ]:
# TODO

### DiffPuter

DiffPuter představuje nejmodernější přístup využívající difuzní modely (známé z generování obrázků) pro imputaci tabulárních dat​​. Jde o kombinaci generativního difuzního modelu s EM algoritmem: model iterativně střídá fázi trénování (M-step), kdy se pomocí difuzního modelu odhaduje společné rozdělení dat včetně chybějících hodnot, a fázi vzorkování (E-step), kdy se chybějící hodnoty aktualizují podmíněným generováním z trénovaného modelu​. Tato kombinace umožňuje postupně zpřesňovat imputace i při vysoké nejistotě. Teoreticky autoři ukazují, že trénovací krok maximalizuje věrohodnost pozorovaných dat a vzorkovací krok odpovídá Bayesovskému odhadu chybějících hodnot​. V testech na 10 různorodých datových souborech DiffPuter překonal 17 existujících metod – průměrně o 8,1 % menší MAE a o 5,6 % menší RMSE než nejlepší porovnávaný algoritmus​. DiffPuter byl publikován jako ICLR 2025 Spotlight; autoři plánují vydat open-source kód (při publikaci na ICLR bývá uvolnění kódu běžné).

In [ ]:
# TODO